# Collaboration and Competition

---

In this notebook, we implement a Multi-Agent Reinforcement Learning solution to the Unity ML-Agents Tennis environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [59]:
from sys import platform
import os.path
import time
import random
from itertools import count
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from typing import List
from unityagents import UnityEnvironment

In [60]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameters to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```
The code below is specific to the servers that were run to train this notebook. Please feel free to edit to ensure it works for your local environment.

In [61]:
if platform.startswith('darwin'):
    env = UnityEnvironment(file_name="./env/Tennis.app")
elif platform.startswith('linux'):
    env = UnityEnvironment(file_name="./env/Tennis_Linux_NoVis/Tennis.x86_64", no_graphics=True)
else:
    raise NotImplemented()

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [62]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [63]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, we demonstrate how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you can watch the agents' performance, if they select actions at random with each time step. If you are using a non-headless environment, a window should pop up that allows you to observe the agents.

In [8]:
# for i in range(1, 6):                                      # play game for 5 episodes
#     env_info = env.reset(train_mode=False)[brain_name]     # reset the environment
#     states = env_info.vector_observations                  # get the current state (for each agent)
#     scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#     while True:
#         actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#         actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#         env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#         next_states = env_info.vector_observations         # get next state (for each agent)
#         rewards = env_info.rewards                         # get reward (for each agent)
#         dones = env_info.local_done                        # see if episode finished
#         scores += env_info.rewards                         # update the score (for each agent)
#         states = next_states                               # roll over states to next time step
#         if np.any(dones):                                  # exit loop if episode finished
#             break
#     print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.09000000171363354
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.09000000171363354


### 4. Implementation

Now we will train our own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
class MultiAgentReplayBuffer(object):
    def __init__(self, max_buffer_size, num_agents, state_shape, action_size):
        """
        Initialize a ReplayBuffer object.
        :param max_buffer_size: maximum size of the buffer
        :param num_agents: number of agents
        :param state_shape: shape of state with first dimension being the number of agents
        :param action_size: size of each action
        """
        self.max_buffer_size = max_buffer_size
        self.num_agents = num_agents
        self.state_shape = state_shape
        self.action_size = action_size
        self.states = np.full((self.max_buffer_size, *self.state_shape), np.nan)
        self.actions = np.full((self.max_buffer_size, self.num_agents, self.action_size), np.nan)
        self.rewards = np.full((self.max_buffer_size, self.num_agents, 1), np.nan)
        self.next_states = np.full((self.max_buffer_size, *self.state_shape), np.nan)
        self.dones = np.full((self.max_buffer_size, self.num_agents, 1), np.nan)
        self.buffer_size = 0
        self.idx = 0

    def __len__(self):
        """
        Return the current size of the buffer.
        :return: current size of the buffer
        """
        return self.buffer_size

    def upsert(self, states, actions, rewards, next_states, dones):
        """
        Insert samples into the buffer and increment the index.
        :param states: np.array of shape (num_agents, state_size)
        :param actions: np.array of shape (num_agents, action_size)
        :param rewards: np.array of shape (num_agents, 1)
        :param next_states: np.array of shape (num_agents, state_size)
        :param dones: np.array of shape (num_agents, 1)
        """
        self.states[self.idx] = states
        self.actions[self.idx] = actions
        self.rewards[self.idx] = rewards
        self.next_states[self.idx] = next_states
        self.dones[self.idx] = dones
        self.buffer_size = min(self.buffer_size + 1, self.max_buffer_size)
        self.idx = (self.idx + 1) % self.max_buffer_size

    def sample(self, batch_size):
        if batch_size > self.buffer_size:
            raise ValueError("Not enough samples in buffer")
        idx = np.random.randint(0, self.idx, batch_size)
        return {
            'states': self.states[idx],
            'states_flat': self.states[idx].reshape(batch_size, self.states[idx].size),
            'actions': self.actions[idx],
            'actions_flat': self.actions[idx].reshape(batch_size, self.actions[idx].size),
            'rewards': self.rewards[idx],
            'next_states': self.next_states[idx],
            'next_states_flat': self.next_states[idx].reshape(batch_size, self.actions[idx].size),
            'dones': self.dones[idx]
        }

In [ ]:
class MATD3TwinCriticNetwork(nn.Module):
    def __init__(self, all_agents_state_size: int, all_agents_action_size: int, hidden_dims: List[int]=(256, 256), activation_fn = F.relu, device=torch.device('cpu')):
        """
        Centralized action-value function approximator that takes as input the actions of all agents and the states of all agents, and outputs the Q-value for each agent.
        :param all_agents_state_size: Dimension of the states of all agents concatenated.
        :param all_agents_action_size: Dimension of the actions of all agents concatenated.
        :param hidden_dims: List of hidden dimensions for the fully connected layers.
        :param activation_fn: Activation function for the fully connected layers.
        :param device: Device on which to run the neural network.
        """
        super(MATD3TwinCriticNetwork, self).__init__()
        self.device = device
        self.all_agents_state_size = all_agents_state_size
        self.all_agents_action_size = all_agents_action_size
        self.activation_fn = activation_fn

        input_dim = self.all_agents_state_size + self.all_agents_action_size
        output_dim = 1

        self.layers_a = nn.ModuleList()
        self.layers_b = nn.ModuleList()
        for i, dim in enumerate(hidden_dims):
            if i == 0:
                self.layers_a.append(nn.Linear(input_dim, dim))
                self.layers_b.append(nn.Linear(input_dim, dim))
            else:
                self.layers_a.append(nn.Linear(hidden_dims[i-1], dim))
                self.layers_b.append(nn.Linear(hidden_dims[i-1], dim))
            self.layers_a.append(activation_fn)
            self.layers_b.append(activation_fn)
        self.layers_a.append(nn.Linear(hidden_dims[-1], output_dim))
        self.layers_b.append(nn.Linear(hidden_dims[-1], output_dim))
        self.layers_a.append(nn.Tanh())
        self.layers_b.append(nn.Tanh())
        self.to(self.device)

    def forward(self, states, actions):
        """
        Takes as input the states and actions of all agents, and outputs the Q-value for each agent.
        :param states: States of all agents concatenated.
        :param actions: Actions of all agents concatenated.
        """
        s, a = self._format(states, actions)
        x = torch.cat([s, a], dim=1)
        for layer_a, layer_b in zip(self.layers_a, self.layers_b):
            xa = layer_a(x)
            xb = layer_b(x)
        return xa, xb

    def q_a(self, states, actions):
        """
        Takes as input the states and actions of all agents, and outputs the Qa-value for each agent.
        :param states: States of all agents concatenated.
        :param actions: Actions of all agents concatenated.
        """
        s, a = self._format(states, actions)
        x = torch.cat([s, a], dim=1)
        for layer_a in self.layers_a:
            xa = layer_a(x)
        return xa

    def _format(self, states, actions):
        s, a = states, actions
        if not isinstance(s, torch.Tensor):
            s = torch.tensor(s,
                             device=self.device,
                             dtype=torch.float32)
        if not isinstance(a, torch.Tensor):
            a = torch.tensor(a,
                             device=self.device,
                             dtype=torch.float32)
        if s.dim() == 1:
            s = s.unsqueeze(0)
        if a.dim() == 1:
            a = a.unsqueeze(0)
        assert s.dim() == 2 and a.dim() == 2
        return s, a

In [ ]:
class MATD3ActorNetwork(nn.Module):
    def __init__(self, state_size: int, action_size: int, hidden_dims=(256,256), activation_fn = F.relu, device=torch.device(self.config['device'])):
        """
        Policy that takes as input the state of an agent and outputs the actions for that agent.
        :param state_size: Dimension of the state of a single agent.
        :param action_size: Dimension of the actions of a single agent.
        :param hidden_dims: List of hidden dimensions for the fully connected layers.
        :param activation_fn: Activation function for the fully connected layers.
        :param device: Device on which to run the neural network.
        """
        super(MATD3ActorNetwork, self).__init__()
        self.device = device
        self.state_size = state_size
        self.action_size = action_size
        self.activation_fn = activation_fn

        input_dim = state_size
        output_dim = action_size

        self.layers = nn.ModuleList()
        for i, dim in enumerate(hidden_dims):
            if i == 0:
                self.layers.append(nn.Linear(input_dim, dim))
            else:
                self.layers.append(nn.Linear(hidden_dims[i-1], dim))
            self.layers.append(activation_fn)
        self.layers.append(nn.Linear(hidden_dims[-1], output_dim))
        self.layers.append(nn.Tanh())
        self.to(device)

    def forward(self, state):
        x = self._format(state)
        for layer in self.layers:
            x = layer(x)
        return x

    def _format(self, state):
        if not isinstance(state, torch.Tensor):
            state = torch.tensor(state,
                                 device=self.device,
                                 dtype=torch.float32)
        if state.dim() == 1:
            state = state.unsqueeze(0)
        assert state.dim() == 2
        return state

In [51]:
class MATD3Agent:
    def __init__(self, state_size, action_size, all_agents_state_size, all_agents_action_size, config):
        self.state_size = state_size
        self.action_size = action_size
        self.all_agents_state_size = all_agents_state_size
        self.all_agents_action_size = all_agents_action_size
        self.config = config

        self.critic_online = MATD3TwinCriticNetwork(self.all_agents_state_size, self.all_agents_action_size, hidden_dims=self.config['critic_hidden_dims'], activation_fn=self.config['critic_activation_fn'], device=torch.device(self.config['device']))
        self.critic_target = MATD3TwinCriticNetwork(self.all_agents_state_size, self.all_agents_action_size, hidden_dims=self.config['critic_hidden_dims'], activation_fn=self.config['critic_activation_fn'], device=torch.device(self.config['device']))
        self.update_critic_target(1.0)

        self.actor_online = MATD3ActorNetwork(self.state_size, self.action_size, hidden_dims=self.config['actor_hidden_dims'], activation_fn=self.config['actor_activation_fn'], device=torch.device(self.config['device']))
        self.actor_target = MATD3ActorNetwork(self.state_size, self.action_size, hidden_dims=self.config['actor_hidden_dims'], activation_fn=self.config['actor_activation_fn'], device=torch.device(self.config['device']))
        self.update_actor_target(1.0)

        self.critic_optimizer = optim.Adam(self.critic_online.parameters(), lr=self.config['critic_lr'])
        self.actor_optimizer = optim.Adam(self.actor_online.parameters(), lr=self.config['actor_lr'])

    def optimize(self, agent_idx: int, experiences, target_next_actions, optimize_policy: bool):
        """
        Optimizes the models of the agent.
        :param agent_idx: Index of this agent.
        :param experiences: Experience sample from the MultiAgentReplayBuffer of all agents.
        :param target_next_actions: Numpy array of shape (num_agents, batch_size, action_size) containing the actions of the next state for each agent from the target policy.
        :param optimize_policy: Whether to optimize the policy or not.
        """

        # online value model loads experiences into device

        batch_size = len(experiences['dones'])
        states = experiences['states']
        states_flat = experiences['states_flat']
        states_self = experiences['states'][:, agent_idx, :]
        actions = experiences['actions']
        actions_flat = experiences['actions_flat']
        rewards_self = experiences['rewards'][:, agent_idx]
        next_states_flat = experiences['next_states_flat']
        dones_self = experiences['dones'][:, agent_idx]

        target_next_actions_flats = target_next_actions.transpose(1,0,2).reshape(batch_size, -1)

        with torch.no_grad():
            target_qa, target_qb = self.critic_target(next_states_flat, target_next_actions_flats)
            target_q = torch.from_numpy(rewards_self) + \
                       torch.from_numpy((1 - dones_self)) * \
                       self.config['gamma'] * \
                       torch.min(target_qa.to('cpu'), target_qb.to('cpu'))
            target_q = target_q.to(self.critic_online.device)

        qa, qb = self.critic_online(states_flat, actions_flat)
        value_loss = F.mse_loss(qa, target_q) + F.mse_loss(qb, target_q)
        self.critic_optimizer.zero_grad()
        value_loss.backward()
        nn.utils.clip_grad_norm(self.critic_online.parameters(), self.config['critic_gradient_clip_value'])
        self.critic_optimizer.step()

        if optimize_policy:
            greedy_action = self.actor_online(states_self)
            actions_copy = np.copy(actions)
            actions_copy[:, agent_idx] = greedy_action.detach().cpu().numpy()
            q_value = self.critic_online.q_a(states, actions_copy.reshape(batch_size, -1))
            policy_loss = -q_value.mean()
            self.actor_optimizer.zero_grad()
            policy_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.actor_online.parameters(), self.config['actor_gradient_clip_value'])
            self.actor_optimizer.step()

    def select_online_action(self, state, noise_std_dev: float, noise_clip_range: float):
        greedy_action = self.actor_online(state).cpu().detach().numpy()
        noise = np.random.normal(0.0, scale=noise_std_dev, size=self.action_size)
        noise = np.clip(noise, -abs(noise_clip_range), abs(noise_clip_range))
        noisy_action = np.clip(greedy_action + noise, -1, 1) # TODO dynamically rescale clip of action to support other environments; currently it's between -1 and 1 only
        return noisy_action

    def select_target_action(self, state, noise_std_dev: float, noise_clip_range: float, use_numpy: bool=True):
        greedy_action = self.actor_target(state)
        noise = torch.randn_like(greedy_action) * noise_std_dev
        noise = torch.clamp(noise, -noise_clip_range, noise_clip_range)
        noisy_action = greedy_action + noise
        noisy_action = torch.clamp(noisy_action, -1, 1) # TODO dynamically rescale clip of action to support other environments; currently it's between -1 and 1 only
        return noisy_action.cpu().detach().numpy() if use_numpy else noisy_action

    def update_critic_target(self, tau: float):
        self._update_target(self.critic_target, self.critic_online, tau)

    def update_actor_target(self, tau: float):
        self._update_target(self.actor_target, self.actor_online, tau)

    def save_checkpoints(self, name: str):
        print("Saving checkpoints for {}".format(name))
        torch.save({
            'critic_online': self.critic_online.state_dict(),
            'critic_optimizer': self.critic_optimizer.state_dict(),
            'actor_online': self.actor_online.state_dict(),
            'actor_optimizer': self.actor_optimizer.state_dict(),
        }, os.path.join(self.config['checkpoint_dir'], '{}.tar'.format(name)))

    def load_checkpoints(self, name: str, train_mode: bool=False):
        message = "Loading checkpoints for {}".format(name)
        if train_mode:
            message += " in train mode"
        else:
            message += " in eval mode"
        print(message)

        checkpoint = torch.load(os.path.join(self.config['checkpoint_dir'], '{}.tar'.format(name)))
        self.critic_online.load_state_dict(checkpoint['critic_online'])
        self.critic_optimizer.load_state_dict(checkpoint['critic_optimizer'])
        self.actor_online.load_state_dict(checkpoint['actor_online'])
        self.actor_optimizer.load_state_dict(checkpoint['actor_optimizer'])

        if train_mode:
            self.critic_online.train()
            self.actor_online.train()
        else:
            self.critic_online.eval()
            self.actor_online.eval()

    @staticmethod
    def _update_target(target, online, tau: float):
        for target_param, param in zip(target.parameters(), online.parameters()):
            target_param.data.copy_(target_param.data * (1.0 - tau) + param.data * tau)

In [6]:
class MATD3Trainer():
    def __init__(self, config, env):
        self.config = config
        torch.manual_seed(self.config['seed']); np.random.seed(self.config['seed']); random.seed(self.config['seed'])

        self.env = env
        self.brain_name = self.env.brain_names[0]
        self.brain = self.env.brains[self.brain_name]
        env_info = self.env.reset(train_mode=True)[self.brain_name]
        self.num_agents = len(env_info.agents)

        self.action_size_per_agent = self.brain.vector_action_space_size
        self.all_agents_state_size = env_info.vector_observations.size
        self.all_agents_action_size = self.action_size_per_agent * self.num_agents

        self.replay_buffer = MultiAgentReplayBuffer(self.config['max_buffer_size'], self.num_agents, env_info.vector_observations.shape, self.action_size_per_agent)

        self.agents = [MATD3Agent(env_info.vector_observations[agent_idx].size, action_size, self.all_agents_state_size, self.all_agents_action_size, self.config) for agent_idx in range(self.num_agents)]

    def train(self):
        self.episode_rewards = []
        self.mean_100_episode_rewards = []
        self.episode_timesteps = []
        self.episode_timestamps = []
        self.episode_durations = []

        training_start = time.time()
        print("Training...")
        for episode in range(1, self.config['max_episodes'] + 1):
            episode_start = time.time()
            env_info = self.env.reset(train_mode=True)[self.brain_name]
            rewards_sum = np.zeros(self.num_agents)

            for step in count(1):
                states = env_info.vector_observations
                actions = [agent.select_online_action(states[agent_idx], self.config['exploration_noise_std_dev'], self.config['exploration_noise_clip_range']) for agent_idx, agent in enumerate(self.agents)]
                actions = np.array(actions)
                env_info = self.env.step(actions)[self.brain_name]
                next_states = env_info.vector_observations
                rewards = env_info.rewards
                dones = env_info.local_done

                self.replay_buffer.upsert(states, actions, rewards, next_states, dones)
                rewards_sum += rewards

                if len(self.replay_buffer) >= self.config['min_buffer_size']:
                    experiences = self.replay_buffer.sample(self.config['batch_size'])
                    noisy_target_next_actions = [agent.select_target_action(experiences['next_states'][:, agent_idx, :], self.config['tps_noise_std_dev'], self.config['tps_noise_clip_range'], use_numpy=True) for agent_idx, agent in enumerate(self.agents)]
                    optimize_policy = step % self.config['optimize_policy_every_steps'] == 0
                    for agent_idx, agent in enumerate(self.agents):
                        agent.optimize(agent_idx, experiences, noisy_target_next_actions, optimize_policy)

                if step % self.config['update_critic_target_every_steps'] == 0:
                    for agent in self.agents:
                        agent.update_critic_target(self.config['tau'])

                if step % self.config['update_actor_target_every_steps'] == 0:
                    for agent in self.agents:
                        agent.update_actor_target(self.config['tau'])

                if np.any(dones):
                    self.episode_timesteps.append(step)
                    self.episode_rewards.append(rewards_sum)
                    episode_end = time.time()
                    self.episode_timestamps.append(episode_end)
                    episode_duration = episode_end - episode_start
                    self.episode_durations.append(episode_duration)
                    break

            mean_100_reward = np.max(self.episode_rewards[-100:], axis=1).mean() if len(self.episode_rewards) >= 100 else np.nan
            goal_reached = mean_100_reward >= self.config['goal_mean_100_reward']

            duration = time.strftime("%H:%M:%S", time.gmtime(time.time() - training_start))
            message = ("{} - episode {:05}, timesteps {:07}, train score 1ep {:.2f}".format(
                duration,
                episode,
                np.sum(self.episode_timesteps, dtype=np.intc),
                np.max(self.episode_rewards[-1:], axis=1).mean(),
                mean_100_reward))
            message += ", train score 100ep {:.2f}".format(mean_100_reward) if mean_100_reward is not np.nan else ""
            print(message)

            if goal_reached:
                print("Goal reached! Training stopped.")
                self.save_checkpoints(episode)
                break

            if episode == self.config['max_episodes']:
                print("Max episodes reached! Training stopped.")
                self.save_checkpoints(episode)
                break

            if episode % self.config['save_every_episodes'] == 0:
                self.save_checkpoints(episode)

    def save_checkpoints(self, episode: int):
        for agent_idx, agent in enumerate(self.agents):
            agent.save_checkpoints('agent{}_episode{:05}'.format(agent_idx, episode))

    def load_checkpoints(self, episode: int, train_mode: bool = False):
        for agent_idx, agent in enumerate(self.agents):
            agent.load_checkpoints('agent{}_episode{:05}'.format(agent_idx, episode), train_mode)

    def demo(self, checkpoint_episode: int):
        self.load_checkpoints(checkpoint_episode, train_mode=False)
        env_info = self.env.reset(train_mode=False)[self.brain_name]
        rewards_sum = np.zeros(self.num_agents)
        while True:
            states = env_info.vector_observations
            actions = [agent.select_online_action(states[agent_idx], 0) for agent_idx, agent in enumerate(self.agents)]
            actions = np.array(actions)
            env_info = self.env.step(actions)[self.brain_name]
            rewards = env_info.rewards
            rewards_sum += rewards
            dones = env_info.local_done
            if np.any(dones):
                break
        return rewards_sum, np.max(rewards_sum, 1)

SyntaxError: EOL while scanning string literal (<ipython-input-6-4239542c8632>, line 81)

In [ ]:
config = {
    'critic_hidden_dims': [256, 256],
    'critic_activation_fn': F.relu,
    'critic_lr': 3e-4,
    'critic_gradient_clip_value': float('inf'),
    'actor_hidden_dims': [256, 256],
    'actor_activation_fn': F.relu,
    'actor_lr': 3e-4,
    'actor_gradient_clip_value': float('inf'),
    'gamma': 0.99,
    'tau': 0.01,
    'checkpoint_dir': os.path.join(os.path.dirname(os.path.realpath('__file__')), 'checkpoints'),
    'max_buffer_size': 1000000,
    'min_buffer_size': 1000,
    'batch_size': 250,
    'max_episodes': 50000,
    'exploration_noise_std_dev': 0.1,
    'exploration_noise_clip_range': float('inf'),
    'tps_noise_std_dev': 0.1,
    'tps_noise_clip_range': 0.5,
    'seed': 42,
    'device': 'cuda:0', # TODO - enable DistributedDataParallel for multi-GPU training
}
matd3 = MATD3Trainer(config, env)

In [39]:
b1 = []
b2 = []
for i in range(1, 6):
    a1 = [-i, i]
    a2 = [-i-.1, i+.1]
    b1.append(a1)
    b2.append(a2)
c = [b1, b2]
print(np.array(c).shape)
c

(2, 5, 2)


[[[-1, 1], [-2, 2], [-3, 3], [-4, 4], [-5, 5]],
 [[-1.1, 1.1], [-2.1, 2.1], [-3.1, 3.1], [-4.1, 4.1], [-5.1, 5.1]]]

In [54]:
c_t = torch.tensor(c)
c_t2 = torch.transpose(c_t, 0, 1).reshape(5, -1)

In [57]:
c_t2.unsqueeze(0).shape

torch.Size([1, 5, 4])

In [86]:
c_t2_np = c_t2.cpu().detach().numpy()
c_t_np = c_t.cpu().detach().numpy()
c_t_np = c_t_np.transpose(1,0,2).reshape(5, -1)

c_t_np == c_t2_np

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])